In [ ]:
wd<-"/result/WGCNA/WT.SCT.res02.domain_220311/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [2]:
save_pheatmap_pdf <- function(x, filename, width=30, height=30
                             ) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename#, width=width, height=height
      )
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}

In [ ]:
library(WGCNA)
library(dplyr)
#enableWGCNAThreads()
enableWGCNAThreads()
options(stringsAsFactors = FALSE)
library(Seurat)
#library(SeuratObject)
library(pheatmap)
#library(ComplexHeatmap)
library(scales)
library(colorspace)

In [5]:
### preprocess: get tissue-domain average gene expression matrix and dattrait2
#load SCT data 
da<-readRDS("/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")

In [ ]:
meta<-da@meta.data
dim(meta)
sctda<-da@assays$SCT@data
dim(sctda)

In [ ]:
sctda[1:5,1:5]
sctda<-as.matrix(sctda)
sctda[1:5,1:5]

In [ ]:
#remove low detection genes
keep<-rowSums(sctda)>=100
table(keep)

In [ ]:
#remove low detection genes
library(edgeR)
y<-DGEList(counts=sctda)
#筛选cpm>1>=2的基因
keep<-rowSums(cpm(y)>1)>=20
y<-y[keep,,keep.lib.sizes=FALSE]
#y<-calcNormFactors(y)
edgrdata=y$counts
dim(edgrdata)

In [ ]:
sctda_filt<-sctda[keep,]
dim(sctda_filt)

In [111]:
sctda_filt<-edgrdata

In [ ]:
unique(meta$domain_res02)
unique(meta$orig.ident)
meta$group<-paste0(meta$orig.ident,"_",meta$domain_res02)
datexpr<-data.frame(row.names = rownames(sctda_filt))
for(i in unique(meta$group)){
    cells<-rownames(subset(meta,group==i))
    datexpr[,i]<-apply(sctda_filt[,cells],1,mean)
}
dim(datexpr)
head(datexpr)

In [198]:
saveRDS(datexpr,"WT.SCT.domain.thr100.datexpr.rds")

In [4]:
datexpr<-readRDS("WT.SCT.domain.thr100.datexpr.rds")

In [ ]:
# transverse expression matrix
datexpr<-as.data.frame(t(as.matrix(datexpr)))
dim(datexpr)
datexpr[1:5,1:5]

In [ ]:
#1.b checking data for excessive missing data and identification of outlier samples
gsg=goodSamplesGenes(datexpr,verbose = 3)
gsg$allOK  #last statement returns TRUE means all genes pass the cuts
#cluster samples to see if there are any obvious outliers
sampletree=hclust(dist(datexpr),method = "average")


In [8]:
#Remove the offending genes and samples from the data:
datexpr<-datexpr[gsg$goodSamples,gsg$goodGenes]

In [ ]:
#library(WGCNA)
options(repr.plot.width=10, repr.plot.height=7#,font.size=1.5
       )
#sampletree=hclust(dist(datexpr),method = "average")
pdf("WT.domain.SCT.data.thr100.average_sample.hclust.pdf",width = 10,height = 7)
#
plot(sampletree,main = "WT.SCT.domain_sampletree.hclustering to detect outliers",
     sub = "",xlab = "",cex=0.7#,cex.axis=1,cex.main=3
    )
dev.off()

In [ ]:
# create dattrait
dattrait=data.frame(row.names =rownames(datexpr))
for(i in 1:nrow(datexpr)){
    dattrait[,rownames(datexpr)[i]]=c(rep(0,i-1),1,rep(0,nrow(datexpr)-i))
}
dattrait
dim(dattrait)

In [11]:
save(datexpr,dattrait,file = "WT.SCT.domain.thr100-01-datainput.RData")

In [6]:
#load data saved in the first part
#rm(list=ls())
lnames=load(file = "WT.SCT.domain.thr100-01-datainput.RData")

In [ ]:

#the variable lnames contains the name of loaded variables
#lnames
#2.a.1 choosing the soft-thresholding power(beta):analysis of network topology
#co-expression similarity is raised to calculate adjacency
#choose a set of soft-thresholding powers
powers=c(1:30)
##call the network topology analysis function
sft=pickSoftThreshold(datexpr,dataIsExpr = TRUE,
                      powerVector = powers,corFnc = cor,
                      corOptions = list(use = 'p'),
                      networkType = "signed")  

In [ ]:
### pick a soft threshold power near the curve of the plot
cex1=0.9
plot(sft$fitIndices[, 1],
     -sign(sft$fitIndices[, 3]) * sft$fitIndices[, 2],
     xlab = "Soft Threshold (power)",
     ylab = "Scale Free Topology Model Fit, signed R^2",
     main = paste("Scale independence")
)
text(sft$fitIndices[, 1],
     -sign(sft$fitIndices[, 3]) * sft$fitIndices[, 2],
     labels = powers, cex = cex1, col = "red"
)
abline(h = 0.80, col = "red")
plot(sft$fitIndices[, 1],
     sft$fitIndices[, 5],
     xlab = "Soft Threshold (power)",
     ylab = "Mean Connectivity",
     type = "n",
     main = paste("Mean connectivity")
)
text(sft$fitIndices[, 1],
     sft$fitIndices[, 5],
     labels = powers,
     cex = cex1, col = "red")

In [23]:
softpower=18
adjacency=adjacency(datexpr,power = softpower,type = "signed")


In [ ]:
# Turn adjacency into topological overlap
TOM = TOMsimilarity(adjacency,TOMType = "signed")
dissTOM = 1-TOM

In [78]:
# Call the hierarchical clustering function
geneTree = hclust(as.dist(dissTOM), method = "average"
                 )

In [ ]:
# Plot the resulting clustering tree (dendrogram)
#sizeGrWindow(12,9)
#pdf("Gene clustering on TOM-based dissimilarity.pdf")
plot(geneTree, xlab="", sub="", main = "Gene clustering on TOM-based dissimilarity",
     labels = FALSE, hang = 0.04)
#dev.off()

abline(h = 0.99,col="red")

In [ ]:
minModuleSize = 30
# Module identification using dynamic tree cut:
dynamicMods = cutreeDynamic(dendro = geneTree, #distM = dissTOM,
                            method = "tree",
                deepSplit = 2,pamRespectsDendro = FALSE,
                #minSplitHeight=0.3,
                #cutHeight=0.99,
                minClusterSize = minModuleSize)
table(dynamicMods)

In [ ]:
# Convert numeric lables into colors
dynamicColors = labels2colors(dynamicMods)
table(dynamicColors)
# Plot the dendrogram and colors underneath
#sizeGrWindow(30,20)
options(repr.plot.width=10,repr.plot.height=7)
#pdf("Dynamic Tree Cut.pdf",width = 10,height = 7)

plotDendroAndColors(geneTree, dynamicColors, "Dynamic Tree Cut",
                    dendroLabels = FALSE, hang = 0.03,
                    addGuide = TRUE, guideHang = 0.05,
                    main = "Gene dendrogram and module colors")
abline(h = 0.99,col="red")
#dev.off()

In [81]:
# Calculate eigengenes
MEList = moduleEigengenes(datexpr, colors = dynamicColors)
MEs = MEList$eigengenes
# Calculate dissimilarity of module eigengenes
MEDiss = 1-cor(MEs);
# Cluster module eigengenes
METree = hclust(as.dist(MEDiss), method = "average");

In [ ]:
# Plot the result
#sizeGrWindow(7, 6)
#pdf("Clustering of module eigengenes.pdf",width = 10,height = 7)
plot(METree, main = "Clustering of module eigengenes",
     xlab = "", sub = "")
MEDissThres = 0.15
# Plot the cut line into the dendrogram
abline(h=MEDissThres, col = "red")
#abline(h=0.1,col="blue")
#dev.off()

In [ ]:
# Call an automatic merging function
merge = mergeCloseModules(datexpr, dynamicColors, cutHeight = MEDissThres, verbose = 3)
# The merged module colors
mergedColors = merge$colors
# Eigengenes of the new merged modules:
mergedMEs = merge$newMEs

In [85]:
# Rename to moduleColors
moduleColors = mergedColors
# Construct numerical labels corresponding to the colors
colorOrder = c("grey", standardColors(50))
moduleLabels = match(moduleColors, colorOrder)-1
MEs = mergedMEs
# Save module colors and labels for use in subsequent parts
#save(MEs, moduleLabels, moduleColors, geneTree, file = "WT.SCT.domain-02-networkConstruction-stepByStep.RData")

In [ ]:
#pdf("geneDendro-mergheight01.pdf", width = 10, height = 7)
plotDendroAndColors(geneTree, cbind(dynamicColors, mergedColors),
                    c("Dynamic Tree Cut", "Merged dynamic"),
                    dendroLabels = FALSE, hang = 0.03,
                    addGuide = TRUE, guideHang = 0.05)
#dev.off()

In [26]:
# output modules
WGCNA.modules.full.gene.list = data.frame(module = moduleColors,
                                          gene = colnames(datexpr))
write.csv(WGCNA.modules.full.gene.list, file = "thr100.sp18.hclust_average.min30.deep2.h015.34modules.full.gene.list.csv",quote=FALSE)

In [ ]:
WGCNA.modules.full.gene.list<-read.csv("thr100.sp18.hclust_average.min30.deep2.h015.34modules.full.gene.list.csv")
head(WGCNA.modules.full.gene.list)
table(WGCNA.modules.full.gene.list$module=="grey")

In [86]:
#### TOMplot
#transfer dissTOM with a power to make moderately strong, connections more visible in the heatmap
plotTOM<-dissTOM^10
#set diagonal to NA for a nicer plot
diag(plotTOM)<-NA

#TOMplot(plotTOM,geneTree,dynamicColors,)

In [ ]:
myheatcol<-rev(viridis_pal(option = "D")(10))#rev(c('#91BFDB','#FEE090','#FC8D59','#D73027'))
myheatcol

In [ ]:
library(scales)
show_col(rev(c('#91BFDB','#FEE090','#FC8D59','#D73027')))
show_col(rev(viridis_pal(option = "D")(10)))

In [ ]:
png("thr100.sp18.min30.deep2.h015.TOMplot_220720.col2.png")
TOMplot(plotTOM,geneTree,
        moduleColors,#dynamicColors#,
        col=myheatcol
       )
dev.off()

In [ ]:
### merge distance dattrait
head(dattrait)


In [ ]:
# Define numbers of genes and samples
nGenes = ncol(datexpr)
nSamples = nrow(datexpr)
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datexpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
head(MEs)

In [ ]:
## merge distance MEs together
mMEs<-MEs
mMEs$domain<-factor(str_sub(rownames(mMEs),-2,-1),levels=c("WM","MG","DH","VH"))
mMEs$time<-factor(str_split(rownames(MEs),"_[H*T]_",simplify = T)[,1],
                   levels = c('WT_sham','WT_3h','WT_24h','WT_72h'))

mMEs$group<-paste0(mMEs$time,"_",mMEs$domain)
#mMEs<-arrange(mMEs,group_by=domain)
#head(mMEs)
mMEs<-mMEs[,-which(colnames(mMEs)%in%c("domain","time"))]
head(mMEs)

In [ ]:
test<-as.matrix(mMEs[,-which(colnames(mMEs)=="group")])
head(test)
group<-mMEs$group
names(group)<-rownames(mMEs)
head(group)

In [ ]:
mergedMEs<-as.data.frame(rowsum(test,group = group))
head(mergedMEs)

In [97]:
mergedMEs<-mergedMEs[c('WT_sham_DH','WT_3h_DH','WT_24h_DH','WT_72h_DH',
                                  'WT_sham_MG','WT_3h_MG','WT_24h_MG','WT_72h_MG',
                                  'WT_sham_VH','WT_3h_VH','WT_24h_VH','WT_72h_VH',
                                  'WT_sham_WM','WT_3h_WM','WT_24h_WM','WT_72h_WM'),]

In [ ]:
dattrait2<-data.frame(row.names = rownames(mergedMEs))
j=1
for(i in rownames(mergedMEs)){
    dattrait2[,i]<-c(rep(0,j-1),1,rep(0,nrow(dattrait2)-j))
    j=j+1
}
dattrait2

In [15]:
moduleTraitCor = cor(mergedMEs, dattrait2, use = "p")
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, 16)

In [ ]:
# Will display correlations and their p-values
pdf("module-time_domain relationships.pdf",width=30,height=30)
textMatrix =  paste(signif(moduleTraitCor, 2), "\n(",
                           signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(13,12, 3, 3));
# Display the correlation values within a heatmap plot
options(repr.plot.width=30,repr.plot.height=30)
labeledHeatmap(Matrix = moduleTraitCor,
               xLabels = names(dattrait2),
               yLabels = names(mergedMEs),
               ySymbols = names(mergedMEs),
               colorLabels = FALSE,
               colors = blueWhiteRed(50),
               textMatrix = textMatrix,
               setStdMargins = FALSE,
               cex.text = 1.5,
               zlim = c(-1,1),
               plotLegend=T,
               main = paste("Module-trait relationships"))
dev.off()

In [ ]:
row_anno<-data.frame(row.names = rownames(moduleTraitCor),"module"=gsub("ME","",rownames(moduleTraitCor)))
row_anno
row_color<-row_anno$module
names(row_color)<-row_anno$module
col_anno<-data.frame(row.names = colnames(moduleTraitCor),"domain"=c(rep("DH",4),rep("MG",4),rep("VH",4),rep("WM",4)),
                    "time"=c(rep(c("sham","3h","24h","72h"),4)))
col_anno
domain_col<-c('#BC3C29A8','#0072B5A8','#E18727A8','#20854EA8')
names(domain_col)<-c("DH", "MG", "VH", "WM")
time_col<-c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF')
names(time_col)<-c("sham","3h","24h","72h")
col_list<-list(module=row_color,domain=domain_col,time=time_col)

In [124]:
gap_col<-c(4,8,12)

In [ ]:
p<-pheatmap(moduleTraitCor,#col=colorRampPalette(rev(brewer.pal(10,"Spectral")))(50),
         scale="none",
          cluster_cols=F,cluster_rows=F,#treeheight_row=10,cutree_rows=7,
         annotation_row=row_anno,
         annotation_col=col_anno,
         border_color="NA",
         fontsize=5,
         annotation_colors=col_list,
         legend=T,gaps_col=gap_col,
         #cellwidth=25,cellheight=25,
         show_rownames=T)
save_pheatmap_pdf(p,filename = "module-time_domain relationships.pheatmap.pdf")

In [ ]:
### merge time into unique domain dattrait
mMEs<-MEs
mMEs$domain<-factor(str_sub(rownames(mMEs),-2,-1),levels=c("WM","MG","DH","VH"))
test<-as.matrix(mMEs[,-which(colnames(mMEs)=="domain")])
head(test)
group<-mMEs$domain
names(group)<-rownames(mMEs)
head(group)

In [ ]:
mergedMEs<-as.data.frame(rowsum(test,group = group))
head(mergedMEs)

In [ ]:
dattrait2<-data.frame(row.names = rownames(mergedMEs))
j=1
for(i in rownames(mergedMEs)){
    dattrait2[,i]<-c(rep(0,j-1),1,rep(0,nrow(dattrait2)-j))
    j=j+1
}
dattrait2

In [145]:
moduleTraitCor = cor(mergedMEs, dattrait2, use = "p")
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, 4)

In [ ]:
# Will display correlations and their p-values
pdf("module-domain relationships.withoutText.pdf",width=8,height=30)
textMatrix =  paste(signif(moduleTraitCor, 2), "\n(",
                           signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(13,12, 3, 3));
# Display the correlation values within a heatmap plot
options(repr.plot.width=8,repr.plot.height=30)
labeledHeatmap(Matrix = moduleTraitCor,
               xLabels = names(dattrait2),
               yLabels = names(mergedMEs),
               ySymbols = names(mergedMEs),
               colorLabels = FALSE,
               colors = blueWhiteRed(50),
               #textMatrix = textMatrix,
               setStdMargins = FALSE,
               cex.text = 1.5,
               zlim = c(-1,1),
               plotLegend=T,
               main = paste("Module-trait relationships"))
dev.off()

### export network to cytoscape

In [ ]:
df<-WGCNA.modules.full.gene.list
df<-df[!df$module=="grey",]
head(df)
unique(df$module)

In [ ]:
probes=colnames(datexpr)
head(probes)
length(probes)

In [20]:
#select modules
modules=unique(df$module)

#select modules probes
#inModule=is.finite(match(mergedColors,modules))
#head(inModule)
#table(inModule)

In [157]:
### export module one by one
sf<-"cytoscape/"
if(!dir.exists(sf))
    dir.create(sf)

for(i in 1:length(modules)){
    module=modules[i]
    inModule=(mergedColors==module)#is.finite(match(dynamicColors,i))
    head(inModule)
    modGenes=probes[inModule]
    length(modGenes)
    # select the corresponding Topological Overlap
    modTOM=TOM[inModule,inModule]
    dim(modTOM)
    dimnames(modTOM)=list(modGenes,modGenes)
    modTOM[1:5,1:5]
    # Export the nerwork into edge and node list files Cytoscape can read
    cyt=exportNetworkToCytoscape(modTOM,
                            edgeFile=paste(sf,"CytoscapeInput-edges-",paste(module,collapse = "-"),".txt",sep = ""),
                            nodeFile=paste(sf,"CytoscapeInput-nodes-",paste(module,collapse = "-"),".txt",sep = ""),
                            weighted=TRUE,
                            threshold=0.02,
                            nodeNames=modGenes,
                            altNodeNames=modGenes,
                            nodeAttr=dynamicColors[inModule])
    top100<-cyt$edgeData %>% top_n(.,wt=weight,n=100)
    write.csv(top100,paste(sf,"CytoscapeInput-edges-",paste(module,collapse = "-"),"wt.top100.csv",sep = ""),row.names=F,quote=F)
    top50<-cyt$edgeData %>% top_n(.,wt=weight,n=50)
    write.csv(top50,paste(sf,"CytoscapeInput-edges-",paste(module,collapse = "-"),"wt.top50.csv",sep = ""),row.names=F,quote=F)
}


In [ ]:
### adjust cyan module(remove Rpl*, Rps* related genes)
module="cyan"
    inModule=which(WGCNA.modules.full.gene.list$module==module)#is.finite(match(dynamicColors,i))
    head(inModule)
    modGenes=probes[inModule]
    length(modGenes)
    # select the corresponding Topological Overlap
    modTOM=TOM[inModule,inModule]
    dim(modTOM)
    dimnames(modTOM)=list(modGenes,modGenes)
    modTOM[1:5,1:5]    

In [ ]:
modTOM<-modTOM[!grepl("Rpl",rownames(modTOM)),!grepl("Rpl",rownames(modTOM))]
dim(modTOM)
head(modTOM)

In [ ]:
modTOM<-modTOM[!grepl("Rps",rownames(modTOM)),!grepl("Rps",rownames(modTOM))]
dim(modTOM)
head(modTOM)

In [51]:
modGenes=rownames(modTOM)

In [52]:
# Export the nerwork into edge and node list files Cytoscape can read
cyt=exportNetworkToCytoscape(modTOM,
                            edgeFile=paste("cytoscape/","CytoscapeInput-edges-",paste(module,collapse = "-"),".txt",sep = ""),
                            nodeFile=paste("cytoscape/","CytoscapeInput-nodes-",paste(module,collapse = "-"),".txt",sep = ""),
                            weighted=TRUE,
                            threshold=0.02,
                            nodeNames=modGenes,
                            altNodeNames=modGenes#,
                            #nodeAttr=WGCNA.modules.full.gene.list$module[inModule]
                            )

    top50<-cyt$edgeData %>% top_n(.,wt=weight,n=50)

In [54]:
write.csv(top50,paste("cytoscape/","CytoscapeInput-edges-",paste("cyan",collapse = "-"),"wt.top50_rmRp.220812.csv",sep = ""),row.names=F,quote=F)

### module seurat score correlation with trait

In [ ]:
module_score<-read.csv("WT.domain.thr100.sp18.hclust_average.min30.deep2.h015.33modules.SCT.score.ex_grey.csv")
rownames(module_score)<-module_score[,1]
module_score<-module_score[,-1]
head(module_score)

In [ ]:
meta<-read.csv("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.SCT.pc20.k50.res02.meta.data.csv")
rownames(meta)<-meta[,1]
head(meta)

In [ ]:
colnames(meta)
unique(meta$time)
unique(meta$domain_res02)
table(rownames(module_score)==rownames(meta))

1. time_domain correlation and pval

In [ ]:
mod_dom_mean<-module_score
mod_dom_mean$time_domain<-paste0(meta$time,"_",meta$domain_res02)
mod_dom_mean<-aggregate(mod_dom_mean[,-which(colnames(mod_dom_mean)=="time_domain")],list(mod_dom_mean$time_domain),mean)
rownames(mod_dom_mean)<-mod_dom_mean[,1]
mod_dom_mean<-mod_dom_mean[,-1]
head(mod_dom_mean)

trait<-data.frame(row.names = rownames(mod_dom_mean))
for(i in rownames(trait)){
    #trait[,i]<-0
    for(j in 1:nrow(trait)){
        trait[j,i]<-ifelse(rownames(trait)[j]==i,1,0)
    }
}
trait

c<-cor(mod_dom_mean,trait)
head(c)
pval<-corPvalueStudent(c,16)
pval
c_long<-melt(c)
c_long$Var2<-factor(c_long$Var2,
                    levels = c('WT_sham_WM','WT_3h_WM','WT_24h_WM','WT_72h_WM',
        'WT_sham_MG','WT_3h_MG','WT_24h_MG', 'WT_72h_MG',
        'WT_sham_DH','WT_3h_DH','WT_24h_DH', 'WT_72h_DH',
        'WT_sham_VH','WT_3h_VH','WT_24h_VH', 'WT_72h_VH')
                    #c('WT_sham_DH','WT_sham_MG','WT_sham_VH','WT_sham_WM',
                             #  'WT_3h_DH','WT_3h_MG','WT_3h_VH','WT_3h_WM',
                             #  'WT_24h_DH','WT_24h_MG','WT_24h_VH','WT_24h_WM',
                             #  'WT_72h_DH','WT_72h_MG','WT_72h_VH','WT_72h_WM'
                             # )
                   )
head(c_long)
pval_long<-melt(pval)
pval_long$c<-c_long$value
pval_long$l<-paste0("(",as.character(round(pval_long$c,2)),")","\n",as.character(round(pval_long$value,2)))
head(pval_long)

In [8]:
c<-c[,c('WT_sham_WM','WT_3h_WM','WT_24h_WM','WT_72h_WM',
        'WT_sham_MG','WT_3h_MG','WT_24h_MG', 'WT_72h_MG',
        'WT_sham_DH','WT_3h_DH','WT_24h_DH', 'WT_72h_DH',
        'WT_sham_VH','WT_3h_VH','WT_24h_VH', 'WT_72h_VH'
        )]

In [9]:
library(reshape2)

In [ ]:
head(c)
c_df<-as.data.frame(c)
c_df$module<-rownames(c_df)
c_df<-melt(c_df)
head(c_df)

In [ ]:
df1 = c_df %>% group_by(.,variable) %>% top_n(.,5,value)
head(df1)
length(unique(df1$module))

In [ ]:
c_ad<-c[rev(unique(df1$module)),]
head(c_ad)

In [ ]:
row_anno<-data.frame(row.names = rownames(c_ad),"module"=rownames(c_ad))
row_anno
row_color<-row_anno$module
names(row_color)<-row_anno$module
col_anno<-data.frame(row.names = colnames(c_ad),"domain"=c(rep("WM",4),rep("MG",4),rep("DH",4),rep("VH",4)),
                    "time"=c(rep(c("sham","3h","24h","72h"),4)))
col_anno
domain_col<-c('#20854EA8','#0072B5A8','#BC3C29A8','#E18727A8')
names(domain_col)<-c("WM", "MG", "DH", "VH")
time_col<-c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF')
names(time_col)<-c("sham","3h","24h","72h")
col_list<-list(module=row_color,domain=domain_col,time=time_col)

In [16]:
gap_col<-c(4,8,12)

In [ ]:
#pdf("module-time_domain relationships.pheatmap_220530.pdf")
options(repr.plot.width=8,repr.plot.height=12)
p<-pheatmap(c_ad,
            col=colorRampPalette(colors = c('#0000ff',
                                            #'#440154FF',
                                            '#fffefe',
                                            '#ff0000'
                                            #'#FDE725FF'
            ))(50),
         scale="none",
         cluster_cols=F,cluster_rows=F,#treeheight_row=10,cutree_rows=7,
         annotation_row=row_anno,
         annotation_col=col_anno,
         border_color="NA",
         fontsize=6,
         annotation_colors=col_list,
         legend=T,gaps_col=gap_col,
         #cellwidth=25,cellheight=25,
         show_rownames=T)
save_pheatmap_pdf(p,filename = "module-time_domain relationships.pheatmap_220721.pdf",width = 8,height = 12)
#dev.off()

In [34]:
temp<-as.matrix(as.data.frame(c_ad)[c("antiquewhite1","antiquewhite2","bisque4","blueviolet","brown2","brown4","chocolate4",
      "coral","cyan","darkolivegreen","darkorange","darkred","darkseagreen4","darkturquoise",
       "firebrick4","indianred3","lavenderblush","lightcoral","lightslateblue","lightsteelblue1",
       "mediumpurple","mediumpurple3","mediumpurple4","mistyrose","navajowhite","palevioletred1",
      "palevioletred2","palevioletred3","pink4","purple","skyblue1","skyblue4","thistle4"),])

In [ ]:
row_anno<-data.frame(row.names = rownames(temp),"module"=rownames(temp))
row_anno
row_color<-row_anno$module
names(row_color)<-row_anno$module
col_anno<-data.frame(row.names = colnames(c_ad),"domain"=c(rep("WM",4),rep("MG",4),rep("DH",4),rep("VH",4)),
                    "time"=c(rep(c("sham","3h","24h","72h"),4)))
col_anno
domain_col<-c('#20854EA8','#0072B5A8','#BC3C29A8','#E18727A8')
names(domain_col)<-c("WM", "MG", "DH", "VH")
time_col<-c('#374E55FF','#DF8F44FF','#00A1D5FF','#B24745FF')
names(time_col)<-c("sham","3h","24h","72h")
col_list<-list(module=row_color,domain=domain_col,time=time_col)

In [ ]:
#pdf("module-time_domain relationships.pheatmap_220530.pdf")
options(repr.plot.width=8,repr.plot.height=12)
p<-pheatmap(temp,
            col=colorRampPalette(colors = c('#0000ff',
                                            #'#440154FF',
                                            '#fffefe',
                                            '#ff0000'
                                            #'#FDE725FF'
            ))(50),
         scale="none",
         cluster_cols=F,cluster_rows=F,#treeheight_row=10,cutree_rows=7,
         annotation_row=row_anno,
         annotation_col=col_anno,
         border_color="NA",
         fontsize=6,
         annotation_colors=col_list,
         legend=T,gaps_col=gap_col,
         #cellwidth=25,cellheight=25,
         show_rownames=T)
save_pheatmap_pdf(p,filename = "module-time_domain relationships.pheatmap_220722.pdf",width = 8,height = 12)
#dev.off()

#### module go term -220804

In [ ]:
modgene<-WGCNA.modules.full.gene.list[WGCNA.modules.full.gene.list$module!="grey",]
length(unique(modgene$module))

In [ ]:
library(clusterProfiler)
options(connectionObserver=NULL)
library(org.Mm.eg.db)
library(ggplot2)

In [ ]:
backid<-read.table("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/WT.GOterm.SCT.backgeneids.txt")
backid<-as.character(backid$x)
head(backid)

In [ ]:
grlabs<-split(modgene$gene,modgene$module)
gcSample<-lapply(grlabs,function(gr) as.numeric(bitr(gr,fromType="SYMBOL",toType = "ENTREZID",OrgDb="org.Mm.eg.db")$ENTREZID))
pvalueCutoff=0.1
qvalueCutoff=0.1
xx.mus.go<-compareCluster(gcSample,OrgDb="org.Mm.eg.db",fun="enrichGO",universe=backid,
                          pvalueCutoff=pvalueCutoff,qvalueCutoff=qvalueCutoff,
                         ont="MF",readable=T)
saveRDS(xx.mus.go,"thr100.sp18.min30.deep2.h015.33modules.module.gene.go.pqval01_MF.220805.rds")

In [4]:
xx.mus.go<-readRDS("thr100.sp18.min30.deep2.h015.33modules.module.gene.go.pqval01_220804.rds")

In [ ]:
df<-xx.mus.go@compareClusterResult
#head(df)
dim(df)
unique(df$Cluster)
df<-df %>% filter(p.adjust<0.05 & qvalue<0.05)
table(df$Cluster)

In [52]:
write.csv(df,"25module.pqval005.ALL.Goterm.dataframe_220805.csv")

In [11]:
library(dplyr)
df1<-df %>% filter(p.adjust<0.05 & qvalue<0.05) %>% group_by(.,Cluster) %>% top_n(.,-3,p.adjust)%>% top_n(.,3,Count) # order by cluster and select each top10 terms
df1$Description<-factor(df1$Description,levels=rev(unique(df1$Description))) #order Description by cluster 

In [ ]:
col1<-unique(as.character(df1$Cluster))
col1
col_darken<-darken(col1,0.15,space = "HLS")

In [ ]:
### barplot
options(repr.plot.width=12,repr.plot.height=15)
ggplot(df1,aes(x=Description,y=-log10(p.adjust),
              fill=Cluster))+
    geom_bar(stat="identity",position = "dodge")+
    coord_flip()+
    scale_fill_manual(values = alpha(col_darken,0.88))+
    theme_bw()+
    #geom_text(aes(label=Count),hjust=1,size=3.5,color="white")+
    theme(axis.text.y.left = element_text(size = 10),panel.background = element_blank(),axis.title.y = element_blank(),
          panel.grid.major = element_blank(),panel.grid.minor=element_blank())
ggsave("25module.pqval005.top3.Goterm.barplot_220810.png",width = 12,height = 15,dpi = 300)
#text(p,0.1,df1$Count,cex = 1)

#### representative three modules' manually selected GO terms -220812

In [ ]:
# cyan(9)
temp<-df[df$Cluster=="cyan" & df$Description %in% c("positive regulation of angiogenesis","response to wounding",
                                                    #"myeloid leukocyte migration",
                                                    "leukocyte chemotaxis","response to interferon-gamma"),]
temp<-temp[order(temp$p.adjust),]
temp$Description<-factor(temp$Description,levels = rev(unique(temp$Description)))
temp

In [ ]:
options(repr.plot.width=6,repr.plot.height=3)
ggplot(temp,aes(x=Description,y=-log10(p.adjust),
              fill=Cluster))+
    geom_bar(stat="identity",position = "dodge")+
    coord_flip()+
    scale_fill_manual(values = alpha(darken("cyan",0.15,space = "HLS"),0.88))+
    theme_bw()+
    #geom_text(aes(label=Count),hjust=1,size=3.5,color="white")+
    theme(axis.text.y.left = element_text(size = 10),panel.background = element_blank(),axis.title.y = element_blank(),
          panel.grid.major = element_blank(),panel.grid.minor=element_blank())
ggsave("cyan.selected.Goterm.barplot_220822.png",width = 6,height = 3,dpi = 300)
#text(p,0.1,df1$Count,cex = 1)

In [ ]:
# darkseagreen4(13)
temp<-df[df$Cluster=="darkseagreen4" & df$Description %in% c("ERK1 and ERK2 cascade",#"regulation of tumor necrosis factor production",
                                                    "response to lipopolysaccharide",
                                                    "cellular response to interleukin-1","tumor necrosis factor production"),]
temp<-temp[order(temp$p.adjust),]
temp$Description<-factor(temp$Description,levels = rev(unique(temp$Description)))
temp

In [ ]:
options(repr.plot.width=6,repr.plot.height=3)
ggplot(temp,aes(x=Description,y=-log10(p.adjust),
              fill=Cluster))+
    geom_bar(stat="identity",position = "dodge")+
    coord_flip()+
    scale_fill_manual(values = alpha(darken("darkseagreen4",0.15,space = "HLS"),0.88))+
    theme_bw()+
    #geom_text(aes(label=Count),hjust=1,size=3.5,color="white")+
    theme(axis.text.y.left = element_text(size = 10),panel.background = element_blank(),axis.title.y = element_blank(),
          panel.grid.major = element_blank(),panel.grid.minor=element_blank())
ggsave("darkseagreen4.selected.Goterm.barplot_220822.png",width = 6,height = 3,dpi = 300)
#text(p,0.1,df1$Count,cex = 1)

In [ ]:
# darkseagreen4(13)
temp<-df[df$Cluster=="darkorange" & df$Description %in% c("synapse organization",#"regulation of tumor necrosis factor production",
                                                    "oxidative phosphorylation",
                                                    "synaptic vesicle cycle","neurotransmitter secretion"),]
temp<-temp[order(temp$p.adjust),]
temp$Description<-factor(temp$Description,levels = rev(unique(temp$Description)))
temp

In [ ]:
options(repr.plot.width=6,repr.plot.height=3)
ggplot(temp,aes(x=Description,y=-log10(p.adjust),
              fill=Cluster))+
    geom_bar(stat="identity",position = "dodge")+
    coord_flip()+
    scale_fill_manual(values = alpha(darken("darkorange",0.15,space = "HLS"),0.88))+
    theme_bw()+
    #geom_text(aes(label=Count),hjust=1,size=3.5,color="white")+
    theme(axis.text.y.left = element_text(size = 10),panel.background = element_blank(),axis.title.y = element_blank(),
          panel.grid.major = element_blank(),panel.grid.minor=element_blank())
ggsave("darkorange.selected.Goterm.barplot_220822.png",width = 6,height = 3,dpi = 300)
#text(p,0.1,df1$Count,cex = 1)